# Question answering con sintesi del contesto

In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="gemma2", temperature=0.)  # (è un modello da 2Gb)
llm.invoke("Tanto và la gatta al lardo...")

'"Tanto va la gatta al lardo che ci lascia lo zampino" è un proverbio italiano.\n\n**Significato:**\n\nSignifica che anche chi cerca di evitare di sporcarsi o di coinvolgersi in qualcosa, alla fine lascia delle tracce del suo passaggio o si lascia coinvolgere. In altre parole, è difficile evitare completamente di lasciare un\'impronta o di essere coinvolti in una situazione.\n\n**Spiegazione:**\n\nL\'immagine è quella di una gatta che va a cercare il lardo (un tipo di pancetta grassa e saporita). Anche se la gatta cerca di muoversi con cautela per non sporcarsi, inevitabilmente lascia delle impronte di zampa.\n\n**Uso:**\n\nSi usa per commentare situazioni in cui qualcuno cerca di evitare una responsabilità o una conseguenza, ma finisce per esserne coinvolto.\n\n**Traduzione letterale:**\n\n"Tanto va la gatta al lardo che ci lascia lo zampino" si traduce letteralmente come "Tanto va la gatta al lardo che lascia l\'impronta di zampa".\n\nSpero che questa spiegazione sia utile!\n'

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://datamasters.it/aziende/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [3]:
all_splits[0]

Document(metadata={'source': 'https://datamasters.it/aziende/', 'title': 'Data Masters per le aziende', 'description': 'La trasformazione digitale della tua azienda inizia ora. Progettiamo percorsi di riqualificazione personalizzati, dedicati sia a team tecnici che alla leadership aziendale.', 'language': 'it-IT'}, page_content='Data Masters per le aziende')

In questo esempio andiamo ad utilizzare nomic-embed-text come modello di embedding. Questo modello lo recuperiamo tramite GPT4All, un'applicazione sviluppata da Nomic, simile a LMStudio che ci mette a disposizione in locale una diversa gamma di modelli disponibili su LangChian utilizzabili sia visivamente sia tramite interfaccia LangChain

È stata una delle prime piattaforme a rende disponbili i modelli in locale con interfaccia grafica anche se, dopo la sua pubblicazione, si è aggiornata poco rispetto a LMStudio che oggi offre un po' più funzionalità.

Ad oggi comunque è ancora largamente utilizzata

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings  # https://www.nomic.ai/gpt4all + pip install gpt4all

vectorstore = Chroma.from_documents(
    documents=all_splits, 
    embedding=GPT4AllEmbeddings(model_name="nomic-embed-text-v1.5.f16.gguf"),
    persist_directory="data/rag"
)  # se il modello non è presente, verrà scaricato alla prima esecuzione

In [5]:
question = "Cosa offre Data Masters alle aziende?"
docs = vectorstore.similarity_search(question, k=2)
print(len(docs))

2


In [6]:
print(docs)

[Document(metadata={'description': 'La trasformazione digitale della tua azienda inizia ora. Progettiamo percorsi di riqualificazione personalizzati, dedicati sia a team tecnici che alla leadership aziendale.', 'language': 'it-IT', 'source': 'https://datamasters.it/aziende/', 'title': 'Data Masters per le aziende'}, page_content='Data Masters per le aziende\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nScrivici su WhatsApp'), Document(metadata={'description': 'La trasformazione digitale della tua azienda inizia ora. Progettiamo percorsi di riqualificazione personalizzati, dedicati sia a team tecnici che alla leadership aziendale.', 'language': 'it-IT', 'source': 'https://datamasters.it/aziende/', 'title': 'Data Masters per le aziende'}, page_content='Data Masters per le aziende\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nScrivici su WhatsApp')]

In [7]:
# creazione di una chain per la sintesi 
# dei documenti / testi selezionati
# in base alla richiesta dell'utente

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate.from_template(
    "Riassumi le seguenti informazioni principali in italiano, riportando tutti i dettagli e le descrizioni: {docs}"
)


# Chain
def format_docs(docs):
    return "\n\n".join(doc.metadata["description"] + " " + doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | llm | StrOutputParser()

# Run
docs = vectorstore.similarity_search(question, k=7)

knowledge_base = chain.invoke(docs)

knowledge_base

"Data Masters offre servizi di **trasformazione digitale** per le aziende. \n\nIl loro focus è sulla creazione di **percorsi di riqualificazione personalizzati** rivolti sia ai **team tecnici** che ai **leader aziendali**.  \n\nL'obiettivo è aiutare le aziende a intraprendere un percorso di digitalizzazione efficace e mirato alle loro specifiche esigenze.\n\n\nPer contattarli, è possibile utilizzare il servizio WhatsApp. \n"

In [8]:
prompt = PromptTemplate.from_template(
    """Date queste informazioni: {docs}
Rispondi a questa domanda in modo diretto e conciso, se non conosci la risposta limitati a dire "non lo so", ecco la domanda: {question}""")

chain = prompt | llm | StrOutputParser()

chain.invoke({
    "docs": knowledge_base,
    "question": question
})

'Data Masters offre servizi di trasformazione digitale, creando percorsi di riqualificazione personalizzati per team tecnici e leader aziendali. \n'

### Hands-on

Create un'unica Chain che svolge l'intero task visto in questo jupyter...